In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import geopandas as gpd
import utils
from utils import util_functions

In [19]:
df_path = 'dataset/state_media_on_social_media_platforms.xlsx'
twitter_dir_path = 'dataset/twitter_accounts_info.csv'
twi_df = pd.read_csv(twitter_dir_path, index_col='user_id')
# loading the dataset
df = pd.read_excel(df_path, index_col='Name (English)')
df = util_functions.data_preprocessor(df)
# Load JSON data from file
with open('assets/recognized_countries.json', 'r') as file:
    data = json.load(file)
recognized_countries = data

with open('assets/equivalent_countries.json', 'r') as file:
    data = json.load(file)
equivalent_countries_dict = data

## preprocess twi_df

In [36]:
twi_df = twi_df.sort_values('followers_count', ascending=False)
twi_df['followers_following_ratio'] = twi_df['followers_count'] / (twi_df['following_count'] + 1e-8)
twi_df

,is_blue_verified,created_at,followers_count,following_count,tweet_num,formatted_created_at,followers_following_ratio
user_id,,,,,,,
1115874631,True,2013-01-24 03:18:59,12997139,76,287263,2013-01-24,1.710150e+05
1115874631,True,2013-01-24 03:18:59,12997138,76,287263,2013-01-24,1.710150e+05
487118986,True,2012-02-09 01:10:18,11993525,78,266973,2012-02-09,1.537631e+05
303862998,True,2011-05-23 15:00:26,6659785,4064,126258,2011-05-23,1.638727e+03
87775422,True,2009-11-05 20:30:10,4164481,157,211312,2009-11-05,2.652536e+04
...,...,...,...,...,...,...,...
931264898,False,2012-11-07 05:04:27,6,12,27,2012-11-07,5.000000e-01
770104757633638405,False,2016-08-29 03:44:16,3,39,522,2016-08-29,7.692308e-02
1358757234637819909,False,2021-02-08 12:39:16,3,27,13,2021-02-08,1.111111e-01


In [39]:
twi_df = twi_df.sort_values('formatted_created_at', ascending=True)
twi_df

,is_blue_verified,created_at,followers_count,following_count,tweet_num,formatted_created_at,followers_following_ratio
user_id,,,,,,,
16482491,False,2008-09-27 04:49:14,1177,614,3350,2008-09-27,1.916938e+00
19140447,True,2009-01-18 09:16:43,391204,1058,73764,2009-01-18,3.697580e+02
23499976,True,2009-03-09 20:58:07,92537,1053,14765,2009-03-09,8.787939e+01
23499976,True,2009-03-09 20:58:07,92536,1053,14765,2009-03-09,8.787844e+01
32366162,False,2009-04-17 10:54:58,190,0,1,2009-04-17,1.900000e+10
...,...,...,...,...,...,...,...
1657332569439088640,False,2023-05-13 10:31:28,194,103,247,2023-05-13,1.883495e+00
1660744054185906207,True,2023-05-22 20:27:01,20972,37,258,2023-05-22,5.668108e+02
1660744054185906207,True,2023-05-22 20:27:01,20973,37,258,2023-05-22,5.668378e+02


In [27]:
twi_df = twi_df.drop_duplicates()
twi_df['created_at'] = pd.to_datetime(twi_df['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')

# Save the new column in the desired format
twi_df['formatted_created_at'] = twi_df['created_at'].dt.strftime('%Y-%m-%d')
twi_df.drop('description', axis=1, inplace=True)

In [ ]:
twi_df = twi_df.drop_duplicates(subset=')
twi_df.info()

## Country Focus news_channels counter map prep

In [3]:
country_focus_count_dict = util_functions.find_country_focus_count_dictionary(df, recognized_countries, equivalent_countries_dict)

c:\src\jupyter_codes\foreign-affairs\utils\util_functions.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  country_list = df['focus group'][idx]


## Name, Owner, Parent Graph

In [4]:
util_functions.create_name_owner_parent_graph(df)

## Rest